In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import backtrader as bt
import datetime

In [2]:
#df = pd.read_csv("../raw_data/merged_data_THETAModel.csv")
#df.dropna(inplace=True)
#df

In [123]:
df_pred = pd.read_csv("../data/predictions/merged_data_ETS_model.csv")
df_pred = df_pred.dropna()
df_pred.drop(columns=['Unnamed: 0'], inplace=True)
df_pred['timestamp'] = pd.to_datetime(df_pred['timestamp'])
df_pred.head(10)

,timestamp,open,high,low,close,volat_ATR,predictions
0,2023-01-02,16610.4,16799.0,16541.2,16666.0,348.494688,16539.674162
1,2023-01-03,16665.9,16774.0,16600.3,16667.2,336.009353,16571.260964
2,2023-01-04,16667.3,16984.6,16645.7,16842.1,336.215828,16883.810018
3,2023-01-05,16842.2,16872.8,16740.4,16823.8,321.657555,16799.579425
4,2023-01-06,16823.8,17030.0,16664.8,16943.8,324.767729,16891.919807
5,2023-01-07,16943.9,16973.6,16900.1,16936.5,306.820034,16896.327002
6,2023-01-08,16936.5,17181.1,16905.0,17124.7,304.625746,16835.519674
7,2023-01-09,17124.6,17387.8,17098.2,17169.7,303.552479,17049.366082
8,2023-01-10,17169.6,17485.4,17140.4,17428.8,306.513016,17075.255404
9,2023-01-11,17428.8,18013.2,17306.4,17935.7,335.104943,17635.080411


In [124]:
df_true = pd.read_csv('../data/raw_data/BTCUSDT_1h.csv')
df_true = df_true.drop(columns=['open', 'close', 'volume'])
df_true = df_true[(df_true['timestamp'] > '2023-01-02') & (df_true['timestamp'] < '2023-06-04')]
df_true['timestamp'] = pd.to_datetime(df_true['timestamp'])
df_true['timestamp'] = df_true['timestamp'] - pd.Timedelta(hours=3)
df_true['timestamp_day'] = df_true['timestamp'].dt.floor('D')

df_true[df_true['timestamp_day'] == '2023-01-08']

,timestamp,high,low,timestamp_day
29229,2023-01-08 00:00:00,16949.3,16905.0,2023-01-08
29230,2023-01-08 01:00:00,16930.0,16908.6,2023-01-08
29231,2023-01-08 02:00:00,16927.4,16912.3,2023-01-08
29232,2023-01-08 03:00:00,16939.5,16923.2,2023-01-08
29233,2023-01-08 04:00:00,16939.4,16930.0,2023-01-08
29234,2023-01-08 05:00:00,16937.2,16929.3,2023-01-08
29235,2023-01-08 06:00:00,16945.9,16927.5,2023-01-08
29236,2023-01-08 07:00:00,16947.5,16938.1,2023-01-08
29237,2023-01-08 08:00:00,16949.9,16933.0,2023-01-08
29238,2023-01-08 09:00:00,16960.0,16918.4,2023-01-08


In [56]:
column_names = df.columns.tolist()
column_names

['Unnamed: 0',
 'timestamp',
 'open',
 'high',
 'low',
 'close',
 'volat_ATR',
 'predictions']

In [57]:
#prediction = 2

In [89]:
solutions = []

In [30]:
df_true[df_true['timestamp_day'] == '2023-02-15']

,timestamp,high,low,timestamp_day
30138,2023-02-15 00:00:00,22275.0,22110.4,2023-02-15
30139,2023-02-15 01:00:00,22238.0,22143.5,2023-02-15
30140,2023-02-15 02:00:00,22248.2,22156.7,2023-02-15
30141,2023-02-15 03:00:00,22190.8,22098.6,2023-02-15
30142,2023-02-15 04:00:00,22157.1,22080.6,2023-02-15
30143,2023-02-15 05:00:00,22120.0,22038.5,2023-02-15
30144,2023-02-15 06:00:00,22123.1,22059.6,2023-02-15
30145,2023-02-15 07:00:00,22110.0,22061.2,2023-02-15
30146,2023-02-15 08:00:00,22137.1,22074.2,2023-02-15
30147,2023-02-15 09:00:00,22148.0,22095.8,2023-02-15


In [41]:
#for index, row in df.iterrows():
 #   close_val = row["close"]
  #  if close_val > prediction == True:
   #     test.append(1)
    

In [76]:
df_true[df_true['timestamp_day'] == '2023-01-08']

,timestamp,high,low,timestamp_day
29226,2023-01-08 00:00:00,16942.0,16929.1,2023-01-08
29227,2023-01-08 01:00:00,16932.4,16922.9,2023-01-08
29228,2023-01-08 02:00:00,16940.4,16925.6,2023-01-08
29229,2023-01-08 03:00:00,16949.3,16905.0,2023-01-08
29230,2023-01-08 04:00:00,16930.0,16908.6,2023-01-08
29231,2023-01-08 05:00:00,16927.4,16912.3,2023-01-08
29232,2023-01-08 06:00:00,16939.5,16923.2,2023-01-08
29233,2023-01-08 07:00:00,16939.4,16930.0,2023-01-08
29234,2023-01-08 08:00:00,16937.2,16929.3,2023-01-08
29235,2023-01-08 09:00:00,16945.9,16927.5,2023-01-08


In [119]:
imput = 1000


for _, row_pred in df_pred.iterrows():
    close_value = row_pred['close']
    prediction_value = row_pred['predictions']
    volat_atr = row_pred["volat_ATR"]
    # high_value = row_pred["high"]
    # low_value = row_pred["low"]
    open_value = row_pred["open"]
    day_stamp = row_pred["timestamp"]
    
    short_stop = volat_atr + open_value
    long_stop = open_value - volat_atr
    tp_mult = 0.002
    
    # Short trade - betting the marked will go down.
    if open_value > prediction_value:
        
        for _, row_true in df_true[df_true['timestamp_day'] == day_stamp].iterrows():
        
            low_value = row_true['low']
            high_value = row_true['high']
            time_stamp = row_true['timestamp']

            # If marked drops below 0.2% of opening value, exit to win
            if open_value - (open_value * tp_mult) >= low_value:
                winn = (imput*0.9994) * (1 + tp_mult) 
                imput = winn
                solutions.append(f"Enter short trade, cut off take profit {time_stamp}, {imput}")
                break
            
            # If marked rises above 0.2% of opening value, exit to mitigate loss
            elif short_stop <= high_value:
                loss = (imput*0.9994)*(1-((volat_atr / open_value)))
                imput = loss
                solutions.append(f"Enter short trade, cut off at short stop level {time_stamp}, {imput}")
                break
            
            # Sell at close
            else:
                take_profit = (close_value / prediction_value) * (imput*0.9994)
                imput = take_profit
                solutions.append(f"Enter short trade {day_stamp}, {imput}")
        
    elif open_value < prediction_value:
        
        if open_value + (open_value * tp_mult) <= high_value:
            winn = (imput*0.9994) * (1 + tp_mult)
            imput = winn
            solutions.append(f"Enter long trade, cut off take profit {day_stamp}, {imput}")
        elif long_stop >= low_value:
            loss = (imput*0.9994)*(1-((volat_atr / open_value)))
            imput = loss
            solutions.append(f"Enter long trade, cut off at long stop level {day_stamp}, {imput}")
        else:
            take_profit = (prediction_value / close_value) * (imput*0.9994)
            imput = take_profit
            solutions.append(f"Enter long trade {day_stamp}, {imput}")
            
    else:
        take_profit = 0
        imput = imput
        solutions.append("skip investment")
        

In [120]:
solutions


['Enter short trade, cut off take profit 2023-01-02 00:00:00, 1001.3987999999999',
 'Enter short trade 2023-01-03 00:00:00, 1006.5920636124744',
 'Enter short trade 2023-01-03 00:00:00, 1011.8122595389766',
 'Enter short trade 2023-01-03 00:00:00, 1017.0595274507409',
 'Enter short trade 2023-01-03 00:00:00, 1022.334007743338',
 'Enter short trade 2023-01-03 00:00:00, 1027.6358415404313',
 'Enter short trade 2023-01-03 00:00:00, 1032.9651706975528',
 'Enter short trade 2023-01-03 00:00:00, 1038.3221378058988',
 'Enter short trade 2023-01-03 00:00:00, 1043.7068861961445',
 'Enter short trade 2023-01-03 00:00:00, 1049.11955994228',
 'Enter short trade 2023-01-03 00:00:00, 1054.5603038654638',
 'Enter short trade 2023-01-03 00:00:00, 1060.0292635378985',
 'Enter short trade 2023-01-03 00:00:00, 1065.5265852867255',
 'Enter short trade 2023-01-03 00:00:00, 1071.0524161979406',
 'Enter short trade 2023-01-03 00:00:00, 1076.6069041203282',
 'Enter short trade 2023-01-03 00:00:00, 1082.190197

In [41]:
print(f"Enter short trade {imput}")

Enter short trade 1175.4401420071652


In [128]:
# initial capital
trade_amount = 1000

sell_threshold = 0.002 # 0.2% change in market drives decision making
buy_cost = 0.0001 # 0.01 %
sell_cost = 0.0005 # 0.05 %

trade_results = {
    'day': [], 
    'balance': [],
    'trade_type': [],
    'success': []
}

def helper_append(results, day, balance, trade_type, success) -> dict:
    results['day'].append(day)
    results['balance'].append(balance)
    results['trade_type'].append(trade_type)
    results['success'].append(success)
    return results

for _, row_day in df_pred.iterrows():
    
    # Params for each row for each day
    day_stamp = row_day['timestamp']
    day_open = row_day['open']
    day_close_true = row_day['close']
    day_close_pred = row_day['predictions']
    day_volat_atr = row_day['volat_ATR']
    
    short_stop_threshold = day_open + day_volat_atr
    long_stop_threshold = day_open - day_volat_atr
    
    ### Short Trade - betting the market will go down
    if day_open > day_close_pred:
        # Apply buy_cost
        trade_amount = trade_amount * (1 - buy_cost)
        
        # iterate over every hour in the day:
        for _, row_hour in df_true[df_true['timestamp_day'] == day_stamp].iterrows():
            # Hourly params for analyis
            hour_low_value = row_hour['low']
            hour_high_value = row_hour['high']
            hour_stamp = row_hour['timestamp']

            # if market drops 0.2% below day_open - sell to win
            if hour_low_value < day_open * (1 - sell_threshold):
                # Update balance and apply sell_cost
                # TODO: check value
                trade_amount = trade_amount * (1 + sell_threshold) * (1 - sell_cost)

                # Append results to dict
                trade_results = helper_append(trade_results, day_stamp, trade_amount, 'short', 'win')
                
                # Print
                print(f'day: {day_stamp} - type: short - win  at {hour_stamp.hour} - balance: {trade_amount: 0.2f}')
                break
                
            # If market rises above short_stop_threshold
            elif hour_high_value > short_stop_threshold:
                # Update balance and apply sell_cost
                trade_amount = trade_amount * (1 - (day_volat_atr / day_open)) * (1 - sell_cost)
                
                # Append results to dict
                trade_results = helper_append(trade_results, day_stamp, trade_amount, 'short', 'lose')
                print(f'day: {day_stamp} - type: short - lose at {hour_stamp.hour} - balance: {trade_amount: 0.2f}')
                break
                
            # If market doesn't meet win/lose condition in 24 hours
            if hour_stamp.hour == 23:
                # Update balance and apply sell_cost
                trade_amount = trade_amount * (day_close_true / day_open) * (1 - sell_cost)
                
                # Append results to dict
                trade_results = helper_append(trade_results, day_stamp, trade_amount, 'short', 'exit')
                print(f'day: {day_stamp} - type: short - exit at {hour_stamp.hour} - balance: {trade_amount: 0.2f}')
                
    ### Long Trade - betting the market will go up
    elif day_close_pred > day_open:
        # Apply buy_cost
        trade_amount = trade_amount * (1 - buy_cost)
        
        # iterate over every hour in the day:
        for _, row_hour in df_true[df_true['timestamp_day'] == day_stamp].iterrows():
            # Hourly params for analyis
            hour_low_value = row_hour['low']
            hour_high_value = row_hour['high']
            hour_stamp = row_hour['timestamp']
            
            # If market rises 0.2% above day_open - exit to win
            if hour_high_value > day_open * (1 + sell_threshold):
                # Update balance and apply sell cost
                trade_amount = trade_amount * (1 + sell_threshold) * (1 - sell_cost)
                
                # Append results to dict
                trade_results = helper_append(trade_results, day_stamp, trade_amount, 'long', 'win')
                print(f'day: {day_stamp} - type: long  - win  at {hour_stamp.hour} - balance: {trade_amount: 0.2f}')
                break
            
            # if market drops below long_stop_threshold
            elif hour_low_value < long_stop_threshold:
                # Update balance and apply sell cost
                trade_amount = trade_amount * (1 - (day_volat_atr / day_open)) * (1 - sell_cost)
                
                # Append results to dict 
                trade_results = helper_append(trade_results, day_stamp, trade_amount, 'long', 'lose')
                print(f'day: {day_stamp} - type: long  - lose at {hour_stamp.hour} - balance: {trade_amount: 0.2f}')
                break                 
            
            # If market doesn't meet win/lose condition in 24 hours
            if hour_stamp.hour == 23:
                # Update balance and apply sell_cost
                trade_amount = trade_amount * (day_close_true / day_open) * (1 - sell_cost)
                
                # Append results to dict
                trade_results = helper_append(trade_results, day_stamp, trade_amount, 'short', 'exit')
                print(f'day: {day_stamp} - type: long - exit at {hour_stamp.hour} - balance: {trade_amount: 0.2f}')
            
    ### Else skip speculation for that day (day_open == pred_close)
    else:
        trade_results = helper_append(trade_results, day_stamp, trade_amount, 'skip', 'skip')
        print(f'day: {day_stamp} - type: skip  - balance: {trade_amount: 0.2f}')

day: 2023-01-02 00:00:00 - type: short - win  at 0 - balance:  1001.40
day: 2023-01-03 00:00:00 - type: short - win  at 15 - balance:  1002.80
day: 2023-01-04 00:00:00 - type: long  - win  at 1 - balance:  1004.20
day: 2023-01-05 00:00:00 - type: short - win  at 1 - balance:  1005.61
day: 2023-01-06 00:00:00 - type: long  - win  at 0 - balance:  1007.01
day: 2023-01-07 00:00:00 - type: short - win  at 10 - balance:  1008.42
day: 2023-01-08 00:00:00 - type: short - exit at 23 - balance:  1019.02
day: 2023-01-09 00:00:00 - type: short - exit at 23 - balance:  1021.09
day: 2023-01-10 00:00:00 - type: short - lose at 20 - balance:  1002.26
day: 2023-01-11 00:00:00 - type: long  - win  at 0 - balance:  1003.66
day: 2023-01-12 00:00:00 - type: short - lose at 2 - balance:  980.79
day: 2023-01-13 00:00:00 - type: long  - win  at 0 - balance:  982.16
day: 2023-01-14 00:00:00 - type: short - win  at 0 - balance:  983.54
day: 2023-01-15 00:00:00 - type: short - win  at 0 - balance:  984.91
day: 

In [ ]:
# initial capital
trade_amount = 1000

sell_threshold = 0.002 # 0.2% change in market drives decision making
buy_cost = 0.0001 # 0.01 %
sell_cost = 0.0005 # 0.05 %

trade_results = {
    'day': [], 
    'balance': [],
    'trade_type': [],
    'success': []
}

# helper for appending
def helper_append(results, day, balance, trade_type, success):
    results['day'].append(day)
    results['balance'].append(balance)
    results['trade_type'].append(trade_type)
    results['success'].append(success)
    return results

# Check to see if it's worth entering the market
for _, row_day in df_pred.iterrows():
    
    # Params for each row for each day
    day_stamp = row_day['timestamp']
    day_open = row_day['open']
    day_close_true = row_day['close']
    day_close_pred = row_day['predictions']
    day_volat_atr = row_day['volat_ATR']
    
    short_stop_threshold = day_open + day_volat_atr
    long_stop_threshold = day_open - day_volat_atr
    
    # Enter market
    if day_open != day_close_pred:
        # Apply buy_cost
        trade_amount = trade_amount * (1 - buy_cost)
        
        # iterate over every hour in the day:
        for _, row_hour in df_true[df_true['timestamp_day'] == day_stamp].iterrows():
            # Hourly params for analyis
            hour_low_value = row_hour['low']
            hour_high_value = row_hour['high']
            hour_stamp = row_hour['timestamp']
            
        ### Shorting the market
        # If market drops 0.2% below open value, sell position


In [113]:
trade_results

{'day': [Timestamp('2023-01-02 00:00:00'),
  Timestamp('2023-01-02 00:00:00'),
  Timestamp('2023-01-02 00:00:00'),
  Timestamp('2023-01-02 00:00:00'),
  Timestamp('2023-01-02 00:00:00'),
  Timestamp('2023-01-03 00:00:00'),
  Timestamp('2023-01-03 00:00:00'),
  Timestamp('2023-01-03 00:00:00'),
  Timestamp('2023-01-03 00:00:00'),
  Timestamp('2023-01-03 00:00:00'),
  Timestamp('2023-01-03 00:00:00'),
  Timestamp('2023-01-05 00:00:00'),
  Timestamp('2023-01-05 00:00:00'),
  Timestamp('2023-01-05 00:00:00'),
  Timestamp('2023-01-05 00:00:00'),
  Timestamp('2023-01-05 00:00:00'),
  Timestamp('2023-01-05 00:00:00'),
  Timestamp('2023-01-05 00:00:00'),
  Timestamp('2023-01-05 00:00:00'),
  Timestamp('2023-01-05 00:00:00'),
  Timestamp('2023-01-05 00:00:00'),
  Timestamp('2023-01-05 00:00:00'),
  Timestamp('2023-01-05 00:00:00'),
  Timestamp('2023-01-05 00:00:00'),
  Timestamp('2023-01-06 00:00:00'),
  Timestamp('2023-01-07 00:00:00'),
  Timestamp('2023-01-07 00:00:00'),
  Timestamp('2023-01-

In [114]:
pd.DataFrame(trade_results).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1293 entries, 0 to 1292
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   day         1293 non-null   datetime64[ns]
 1   balance     1293 non-null   float64       
 2   trade_type  1293 non-null   object        
 3   success     1293 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 40.5+ KB
